In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
df_train = pd.read_csv('trainee_train.csv', index_col = 'Unnamed: 0')
df_train

,im,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v437,v438,v439,v440,v441,v442,v443,v444,v445,v446
0,0.0,128.100243,128.157072,129.255102,128.273006,126.532819,129.023232,129.854839,129.828431,129.868526,...,128.800985,128.232694,128.211893,130.512476,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
1,1.0,128.100243,128.157072,126.636364,128.273006,130.810403,129.023232,128.633333,128.994152,128.994152,...,128.914365,128.232694,128.211893,128.167519,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
2,1.0,128.100243,128.157072,129.255102,128.273006,130.810403,129.023232,126.769231,126.926295,127.196460,...,128.800985,128.232694,128.211893,128.167519,128.051724,127.709786,127.349282,127.063052,125.169118,125.274194
3,1.0,128.100243,128.157072,127.630027,128.273006,126.532819,129.023232,126.769231,126.926295,127.196460,...,128.800985,130.794007,131.026119,130.512476,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
4,1.0,128.100243,128.336364,129.255102,128.273006,130.810403,126.042980,129.854839,126.926295,127.196460,...,128.800985,130.794007,131.026119,130.512476,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,0.0,128.100243,128.157072,129.255102,128.273006,130.810403,129.023232,129.854839,129.828431,129.868526,...,128.800985,128.232694,128.211893,126.744108,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
612,1.0,128.100243,128.157072,129.255102,128.273006,126.532819,129.023232,126.769231,126.926295,127.196460,...,128.800985,128.232694,128.211893,126.744108,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
613,1.0,130.798507,131.174905,129.255102,117.900000,132.860465,130.381148,129.854839,129.828431,129.868526,...,128.914365,128.232694,128.211893,128.167519,128.051724,127.709786,127.349282,127.063052,125.169118,126.611554
614,1.0,128.100243,128.157072,129.255102,128.273006,126.532819,129.023232,129.854839,126.926295,127.196460,...,128.800985,128.232694,128.211893,128.167519,128.051724,127.709786,127.349282,127.063052,130.477398,130.496150


In [3]:
df_test = pd.read_csv('trainee_test_fish.csv', index_col = 'Unnamed: 0')
df_test

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v437,v438,v439,v440,v441,v442,v443,v444,v445,v446
0,130.798507,131.174905,129.255102,132.199234,128.746479,130.381148,129.854839,129.828431,129.868526,130.917969,...,128.800985,128.232694,128.211893,128.167519,126.100000,127.709786,127.349282,127.063052,127.825000,126.611554
1,128.100243,128.157072,127.630027,128.273006,130.810403,129.023232,126.769231,126.926295,127.196460,127.433333,...,128.800985,130.794007,127.750000,130.512476,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
2,128.100243,128.157072,129.255102,128.273006,130.810403,126.042980,129.854839,129.828431,129.868526,128.618063,...,128.800985,130.794007,131.026119,130.512476,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
3,130.798507,131.174905,129.255102,132.199234,132.860465,130.381148,129.854839,129.828431,129.868526,130.917969,...,126.503759,128.232694,128.211893,128.167519,128.051724,127.709786,127.349282,127.063052,127.825000,126.611554
4,130.798507,131.174905,129.255102,132.199234,130.810403,130.381148,129.854839,129.828431,129.868526,130.917969,...,128.914365,128.232694,128.211893,128.167519,128.051724,127.709786,127.349282,127.063052,125.169118,126.611554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,130.798507,131.174905,129.255102,117.900000,132.860465,130.381148,129.854839,129.828431,129.868526,130.917969,...,128.914365,128.232694,128.211893,128.167519,128.051724,127.709786,127.349282,127.063052,127.825000,126.611554
212,128.100243,128.157072,126.636364,128.273006,130.810403,129.023232,128.633333,128.994152,128.994152,128.618063,...,128.800985,128.232694,128.211893,126.744108,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
213,128.100243,128.157072,129.255102,128.273006,126.532819,129.023232,127.720000,127.428571,127.428571,128.618063,...,128.914365,128.232694,128.211893,128.167519,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
214,128.100243,128.157072,129.255102,128.273006,126.532819,126.042980,129.854839,129.828431,129.868526,128.618063,...,128.914365,128.232694,128.211893,128.167519,128.051724,127.709786,127.349282,127.063052,130.477398,130.496150


In [4]:
# проверка наличия пропущенных значений в каждом столбце train df
missing_values_train = df_train.isna().sum()

columns_with_missing_values_train = missing_values_train[missing_values_train > 0]
print(columns_with_missing_values_train)

Series([], dtype: int64)


In [5]:
# или то же самое, но вот так на тесте
df_test[df_test.isna().any(axis = 1)]

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v437,v438,v439,v440,v441,v442,v443,v444,v445,v446


In [6]:
df_train.describe()

,im,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v437,v438,v439,v440,v441,v442,v443,v444,v445,v446
count,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000,...,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000
mean,0.717532,128.609606,128.603800,128.660703,128.523063,128.410981,128.610633,128.636522,128.688810,128.761940,...,128.645251,128.532595,128.544489,128.514329,128.514597,128.491559,128.496883,128.459309,128.488060,128.478922
std,0.450565,1.019311,1.059058,0.938667,2.291477,2.232095,1.578053,1.359686,1.332505,1.249895,...,0.685546,0.823381,0.968617,1.358473,1.658867,1.596935,1.965948,1.862891,2.284923,2.158584
min,0.000000,128.100243,127.666667,126.636364,113.000000,126.532819,126.042980,126.769231,126.926295,127.196460,...,126.503759,128.232694,127.750000,120.000000,126.100000,125.223301,125.535433,127.063052,125.169118,125.274194
25%,0.000000,128.100243,128.157072,127.630027,128.273006,126.532819,129.023232,126.769231,126.926295,127.196460,...,128.800985,128.232694,128.211893,128.167519,128.051724,127.709786,127.349282,127.063052,125.169118,126.611554
50%,1.000000,128.100243,128.157072,129.255102,128.273006,126.532819,129.023232,129.854839,129.828431,129.868526,...,128.800985,128.232694,128.211893,128.167519,128.051724,127.709786,127.349282,127.063052,130.477398,130.496150
75%,1.000000,128.100243,128.157072,129.255102,128.273006,130.810403,129.023232,129.854839,129.828431,129.868526,...,128.914365,128.232694,128.211893,130.512476,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
max,1.000000,130.798507,131.174905,129.267241,132.199234,132.860465,136.222222,129.854839,129.828431,129.868526,...,129.252747,130.794007,132.000000,130.512476,130.411160,130.650165,134.210526,131.017268,130.666667,130.496150


In [7]:
# проверка на разрежженность данных
sparse_columns = (df_train.isnull().sum() / len(df_train)) > 0.5
sparse_columns[sparse_columns == True]

Series([], dtype: bool)

In [8]:
# Проверка на мультиколлинеарность

# порог корреляции
threshold = 0.8

correlation_matrix = df_train.corr()

# маска для признаков, где корреляция выше порога
high_correlation_mask = (correlation_matrix.abs() > threshold) & (correlation_matrix.abs() < 1.0)

# фильтруем только признаки с высокой корреляцией
high_correlation_features = correlation_matrix[high_correlation_mask].stack().reset_index()
high_correlation_features.columns = ['Feature 1', 'Feature 2', 'Correlation']

print(high_correlation_features)

     Feature 1 Feature 2  Correlation
0           v1        v2     0.881356
1           v2        v1     0.881356
2           v4       v11     0.971146
3           v5       v15     0.906197
4           v5       v35     0.899064
...        ...       ...          ...
1739      v444      v441     0.806347
1740      v444      v443     0.940316
1741      v445      v446     0.936260
1742      v446      v373     0.808825
1743      v446      v445     0.936260

[1744 rows x 3 columns]


In [9]:
# список уникальных признаков с высокой корреляцией
features_to_drop = set(high_correlation_features['Feature 1']).intersection(set(high_correlation_features['Feature 2']))

# удаляем лишние признаки из матрицы 
df_train_filt = df_train.drop(columns = features_to_drop)

df_train_filt

,im,v3,v6,v7,v10,v12,v13,v14,v18,v19,...,v427,v428,v429,v430,v431,v435,v436,v437,v440,v442
0,0.0,129.255102,129.023232,129.854839,128.618063,127.510710,125.697778,128.359865,129.221007,128.564397,...,130.708696,128.728621,131.214953,128.668153,129.008397,128.022282,132.269231,128.800985,130.512476,130.650165
1,1.0,126.636364,129.023232,128.633333,128.618063,129.663657,128.761221,128.359865,129.221007,128.564397,...,128.131524,128.728621,127.980583,128.817680,129.008397,128.022282,127.792271,128.914365,128.167519,130.650165
2,1.0,129.255102,129.023232,126.769231,128.618063,127.510710,128.761221,127.181208,126.150588,128.564397,...,128.131524,128.728621,127.980583,128.668153,129.008397,128.022282,127.792271,128.800985,128.167519,127.709786
3,1.0,127.630027,129.023232,126.769231,127.433333,129.663657,128.761221,128.359865,129.221007,128.564397,...,129.375648,128.750605,129.990260,128.668153,129.008397,130.867133,130.713355,128.800985,130.512476,130.650165
4,1.0,129.255102,126.042980,129.854839,128.618063,127.510710,128.761221,127.181208,126.150588,128.564397,...,129.375648,128.750605,129.990260,128.668153,129.008397,130.867133,130.713355,128.800985,130.512476,130.650165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,0.0,129.255102,129.023232,129.854839,128.618063,127.510710,128.761221,128.359865,129.221007,128.564397,...,130.708696,128.728621,127.980583,128.668153,129.008397,128.022282,127.792271,128.800985,126.744108,130.650165
612,1.0,129.255102,129.023232,126.769231,128.618063,129.663657,128.761221,128.359865,129.221007,128.564397,...,128.131524,128.728621,127.980583,128.668153,129.008397,128.022282,127.792271,128.800985,126.744108,130.650165
613,1.0,129.255102,130.381148,129.854839,130.917969,130.871795,131.408730,133.642857,130.585000,133.224242,...,128.131524,128.728621,127.980583,128.817680,128.240000,128.022282,127.792271,128.914365,128.167519,127.709786
614,1.0,129.255102,129.023232,129.854839,128.618063,127.510710,128.761221,128.359865,129.221007,128.564397,...,128.131524,128.728621,127.980583,128.668153,129.008397,128.022282,127.792271,128.800985,128.167519,127.709786


Все равно датафрейм кажется излишне большим, я бы понизила размерность

In [10]:
target_train = df_train_filt['im'] 
features_train = df_train_filt.drop(columns = ['im'])

In [11]:
# стандартизация данных, так как важно для метода PCA
scaler = StandardScaler()
features_scaled_train = scaler.fit_transform(features_train)

# применим PCA
pca = PCA(n_components = 0.95)  # тут доля объясненной дисперсии, котрую мы хотим сохранить
features_pca_train = pca.fit_transform(features_scaled_train)

print("Размерность признаков после применения PCA для трейна:", features_pca_train.shape)

Размерность признаков после применения PCA для трейна: (616, 85)


In [12]:
# предобрабоатем тестовую выборку
df_test_filt = df_test.drop(columns = features_to_drop)
features_scaled_test = scaler.transform(df_test_filt)
features_pca_test = pca.transform(features_scaled_test)

print("Размерность признаков после применения PCA для теста:", features_pca_test.shape)

Размерность признаков после применения PCA для теста: (216, 85)


In [74]:
X_train, X_val, y_train, y_val = train_test_split(features_pca_train, target_train, test_size = 0.2, random_state = 17)

In [75]:
svm_classifier_div = SVC()
svm_classifier_div.fit(X_train, y_train)

# прогнозирование на валидации
y_pred_val = svm_classifier_div.predict(X_val)

accuracy_val = accuracy_score(y_val, y_pred_val)
print("Accuracy on validation data:", accuracy_val)


Accuracy on validation data: 0.9032258064516129


In [76]:
# Вычисляем ROC AUC на основе предсказаний классов
roc_auc_1 = roc_auc_score(y_val, y_pred_val)

print("ROC AUC для модели SVC с лучшими параметрами на валидационных данных:", roc_auc_1)

ROC AUC для модели SVC с лучшими параметрами на валидационных данных: 0.8709677419354838


In [78]:
%%time
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly']
}

grid_search_1 = GridSearchCV(svm_classifier_div, param_grid, cv = 5)

grid_search_1.fit(X_train, y_train)

best_params_svm_1 = grid_search_1.best_params_

print("Лучшие параметры для SVM:", best_params_svm_1)

best_svm_classifier_1 = grid_search_1.best_estimator_

Лучшие параметры для SVM: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
Wall time: 9min 8s


In [79]:
best_svm_classifier_1.fit(X_train, y_train)

y_pred_val_1 = best_svm_classifier_1.predict(X_val)

accuracy_val_1 = accuracy_score(y_val, y_pred_val_1)
print("Accuracy на валидации с лучшими параметрами:", accuracy_val_1)

roc_auc_2 = roc_auc_score(y_val, y_pred_val_1)

print("ROC AUC для модели SVC с лучшими параметрами на валидационных данных:", roc_auc_2)

Accuracy на валидации с лучшими параметрами: 0.9032258064516129
ROC AUC для модели SVC с лучшими параметрами на валидационных данных: 0.8709677419354838


In [22]:
svm_classifier = SVC()
svm_classifier.fit(features_pca_train, target_train)

# производительность модели оценим с помощью кросс-валидации
scores = cross_val_score(svm_classifier, features_pca_train, target_train, cv = 5) 
print("Средняя точность на обучающих данных для SVM:", scores.mean())

Средняя точность на обучающих данных для SVM: 0.8782848151062156


In [23]:
predictions_0 = cross_val_predict(svm_classifier, features_pca_train, target_train, cv=5)

# Вычисляем ROC AUC на основе предсказаний классов
roc_auc_3 = roc_auc_score(target_train, predictions_0)

print("ROC AUC для модели SVC на обучающих данных:", roc_auc_3)

ROC AUC для модели SVC на обучающих данных: 0.8175872471004317


In [74]:
%%time
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly']
}

svm_classifier = SVC()

grid_search = GridSearchCV(svm_classifier, param_grid, cv = 5)

grid_search.fit(features_pca_train, target_train)

best_params_svm = grid_search.best_params_

print("Лучшие параметры для SVM:", best_params_svm)

# Оценка производительности модели с лучшими параметрами
best_svm_classifier = grid_search.best_estimator_
scores_svm_best = cross_val_score(best_svm_classifier, features_pca_train, target_train, cv = 5)
print("Средняя точность на обучающих данных для SVM с лучшими параметрами:", scores_svm_best.mean())

Лучшие параметры для SVM: {'C': 1, 'gamma': 0.01, 'kernel': 'poly'}
Средняя точность на обучающих данных для SVM с лучшими параметрами: 0.8815499606608969
Wall time: 6min 32s


In [112]:
# Получаем предсказания классов с помощью кросс-валидации
predictions = cross_val_predict(best_svm_classifier, features_pca_train, target_train, cv=5)

# Вычисляем ROC AUC на основе предсказаний классов
roc_auc = roc_auc_score(target_train, predictions)

print("ROC AUC для модели SVC с лучшими параметрами на обучающих данных:", roc_auc)

ROC AUC для модели SVC с лучшими параметрами на обучающих данных: 0.8163650075414782


In [80]:
final_svm_classifier = SVC(**best_params_svm_1)
# final_svm_classifier = SVC(C = 10, gamma = 0.001, kernel= 'rbf')

# final_svm_classifier.fit(features_pca_train, target_train)
final_svm_classifier.fit(X_train, y_train)

# прогноз на тестовой выборке
y_pred_test = final_svm_classifier.predict(features_pca_test)

In [81]:
res_df = pd.DataFrame({'': range(len(y_pred_test)), 'im': y_pred_test})
res_df.to_csv('vk_predictions.csv', index=False)

Тут просто эксперименты с другими моделями:

In [114]:
lgbm_classifier = LGBMClassifier()
lgbm_classifier.fit(features_pca_train, target_train)

scores_lgbm = cross_val_score(lgbm_classifier, features_pca_train, target_train, cv = 5)  # cv=5 указывает на 5-кратную кросс-валидацию

print("Средняя точность на обучающих данных для LGB:", scores_lgbm.mean())

# Получаем предсказания классов с помощью кросс-валидации
predictions = cross_val_predict(lgbm_classifier, features_pca_train, target_train, cv=5)

# Вычисляем ROC AUC на основе предсказаний классов
roc_auc = roc_auc_score(target_train, predictions)

print("ROC AUC для модели LGB с лучшими параметрами на обучающих данных:", roc_auc)

Средняя точность на обучающих данных для LGB: 0.8474036191974823
ROC AUC для модели LGB с лучшими параметрами на обучающих данных: 0.766474228948874


In [58]:
param_grid = {
    'num_leaves': [31, 50, 100],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300]
}

lgbm_classifier_b = LGBMClassifier()
grid_search = GridSearchCV(lgbm_classifier_b, param_grid, cv = 5)
grid_search.fit(features_pca_train, target_train)

best_lgbm_classifier = grid_search.best_estimator_
scores_best_lgb = cross_val_score(best_lgbm_classifier, features_pca_train, target_train, cv = 5)
print("Средняя точность на обучающих данных для LGB с лучшими параметрами:", scores_best_lgb.mean())

Средняя точность на обучающих данных для LGB с лучшими параметрами: 0.852268554943614


In [115]:
xgb_classifier = XGBClassifier()
xgb_classifier.fit(features_pca_train, target_train)

scores_xgb = cross_val_score(xgb_classifier, features_pca_train, target_train, cv = 5)
print("Средняя точность на обучающих данных для XGB:", scores_xgb.mean())

# Получаем предсказания классов с помощью кросс-валидации
predictions = cross_val_predict(xgb_classifier, features_pca_train, target_train, cv=5)

# Вычисляем ROC AUC на основе предсказаний классов
roc_auc = roc_auc_score(target_train, predictions)

print("ROC AUC для модели XGB с лучшими параметрами на обучающих данных:", roc_auc)


Средняя точность на обучающих данных для XGB: 0.8506556517178074
ROC AUC для модели XGB с лучшими параметрами на обучающих данных: 0.777448379882457


In [116]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(features_pca_train, target_train)

scores_rf = cross_val_score(rf_classifier, features_pca_train, target_train, cv = 5)
print("Средняя точность на обучающих данных для RF:", scores_rf.mean())

# Получаем предсказания классов с помощью кросс-валидации
predictions = cross_val_predict(rf_classifier, features_pca_train, target_train, cv=5)

# Вычисляем ROC AUC на основе предсказаний классов
roc_auc = roc_auc_score(target_train, predictions)

print("ROC AUC для модели RF с лучшими параметрами на обучающих данных:", roc_auc)


Средняя точность на обучающих данных для RF: 0.8328088119590873
ROC AUC для модели RF с лучшими параметрами на обучающих данных: 0.7110833723409788
